In [1]:
import pandas as pd
import os, openai
import time
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings


import string
LETTERS = string.ascii_uppercase

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))

df = pd.read_parquet('data/articles.parquet.gzip')
df.columns = ["src","content","LEN"]
df = df[(df.LEN > 1500) & (df.LEN < 30000)].reset_index(drop=True)
print(len(df),"articles of good lengths in the articles.partquet.gzip")
titles = pd.read_parquet("data/titles.parquet.gzip")
df = df.merge(titles, on="src",how="left")
mt = pd.read_parquet("data/metatags.parquet.gzip")
df = df.merge(mt,on="src",how="left")
df.to_parquet("data/consolidated.parquet.gzip",compression="gzip")
#df["text"] = df.content
df["source"] = df.url
for x in df.columns:
    df[x] = df[x].astype(str)
df_loader = DataFrameLoader(df, page_content_column="content")

df_document = df_loader.load()


OPENAI_API_KEY is ready


/tmp/ipykernel_28908/795153844.py:7: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


2039 articles of good lengths in the articles.partquet.gzip


In [3]:
df

,src,content,LEN,title,keywords,themes,summary,origin,url,source
0,f9ef1fc984ab921e2f13d2c53db0a066,Google adopts small nuclear power reactors at ...,3339,Google Embraces Small Nuclear Reactors,['Google' 'nuclear power' 'Kairos Power' 'AI' ...,['Technology' 'Energy' 'Artificial Intelligence'],Google has entered an agreement with nuclear p...,20241027,https://www.tomshardware.com/tech-industry/art...,https://www.tomshardware.com/tech-industry/art...
1,62cfefe39b246a28297152e2e5401c44,"Introduction\nEvery year, in latitudes far eno...",10932,Cyanobacteria's Seasonal Anticipation Abilities,['cyanobacteria' 'circadian clock' 'seasonal r...,['Biology' 'Microbiology' 'Evolution'],"The text discusses research on cyanobacteria, ...",20241110,https://www.quantamagazine.org/even-a-single-b...,https://www.quantamagazine.org/even-a-single-b...
2,bcb3358ea987c50fb056866ca37eef83,LOS ANGELES — “Maybe ‘apocalypse’ is the oppor...,5041,Indigenous Futures Through Art Exhibition,['Indigenous' 'art' 'sculpture' 'technology' '...,['Indigenous Culture' 'Contemporary Art' 'Tech...,"The text discusses the exhibition ""Future Imag...",20241027,https://hyperallergic.com/960383/from-the-ruin...,https://hyperallergic.com/960383/from-the-ruin...
3,27e2e93d576270c6eaee2cfd32e5141c,Most of the passengers emerging from the stati...,6825,Barcelona's Innovative Subway Energy Solutions,['Barcelona' 'regenerative braking' 'MetroCHAR...,['Sustainable Transportation' 'Renewable Energ...,The text discusses Barcelona's innovative subw...,20241110,https://grist.org/transportation/barcelona-is-...,https://grist.org/transportation/barcelona-is-...
4,e48efbb7b17be575d661d0fe6065fd05,- Unsupervised Learning\n- Posts\n- How to Use...,2104,Using Hugging Face with Ollama,['Ollama' 'Hugging Face' 'models' 'AI' 'CLI' '...,['Artificial Intelligence' 'Machine Learning' ...,The text provides a step-by-step guide on how ...,20241110,https://danielmiessler.com/p/how-to-use-huggin...,https://danielmiessler.com/p/how-to-use-huggin...
...,...,...,...,...,...,...,...,...,...,...
2034,edbb7336dfb26098bcc966ca17c074b9,It was time to consider calling it a night. Sh...,15656,The Radical Power of Hanging Out,['hanging out' 'book' 'Sheila Liming' 'crisis'...,['socializing' 'isolation' 'technology'],"The text revolves around the concept of ""hangi...",20230305,https://slate.com/culture/2023/02/hanging-out-...,https://slate.com/culture/2023/02/hanging-out-...
2035,3fbb11d6e949d1e662aa6a146bb6cda0,China’s growing number of insomniacs are turni...,2859,China's Insomniacs Turn to Sleeping Assistants,['insomniacs' 'sleeping assistants' 'Taobao' '...,['Insomnia' 'Sleeping services' 'Sleep economy'],"China has a growing number of insomniacs, lead...",20221031,https://www.sixthtone.com/news/1011417?utm_sou...,https://www.sixthtone.com/news/1011417?utm_sou...
2036,e8b7faf708568f9f39abe04b778c4631,I would say my efficiency is up ~20% since sta...,14337,The Impact of ChatGPT on Efficiency and Inform...,['efficiency' 'Google searches' 'StackOverflow...,['productivity enhancement' 'automation' 'reli...,The text highlights the author's increased eff...,20230521,https://news.ycombinator.com/item?id=35846386,https://news.ycombinator.com/item?id=35846386
2037,b79a2baa87b68283198416791b93bce4,The U.S. government has restricted sales of Nv...,2125,US Restricts Nvidia GPU Sales to Thwart China'...,['U.S. government' 'Nvidia' 'high-performance ...,['Export regulations' 'China-US relations' 'AI...,The U.S. government has imposed restrictions o...,20230320,https://www.tomshardware.com/news/us-bans-sale...,https://www.tomshardware.com/news/us-bans-sale...


In [5]:

text_splitter = CharacterTextSplitter(separator='\n\n',chunk_size=2000, chunk_overlap=200)
chunked_documents = text_splitter.split_documents(df_document)
print(len(chunked_documents),"docs to add")

base_path = "./DB/"
if os.getenv("OAI") is not None:
    openai.api_key = os.getenv("OAI")
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

embeddings = OpenAIEmbeddings(openai_api_key=os.getenv("OAI"))


if not os.path.isfile(base_path+"chroma.sqlite3"):
    print("Start a new DB")
    vectordb = Chroma.from_documents(
        documents=[chunked_documents[0]],
        embedding=embeddings,
        persist_directory=base_path
    )
    vectordb.persist()
else:
    print("Continue on the DB")
    vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
    print(len(vectordb.get()["ids"]),"elements already stored.")
    LSDOCS = vectordb.get()["documents"]

print("Already",len(vectordb.get()["documents"]),"documents.")
print("Adding",len(chunked_documents),"documents.")

if vectordb:
    LSDOCS = vectordb.get()["documents"]
else:
    LSDOCS = []
    
for doc in chunked_documents:
    # Check if the text already exists somewhere
    if not doc.page_content in LSDOCS:
        vectordb.add_documents(
            documents=[doc], 
            embedding=embeddings, 
            persist_directory=base_path
        )
        # Ugly hack to avoid reaching token per min limit 
        # So it sleeps 1s between page
        time.sleep(0.001)
        vectordb.persist()
    else:
        0
        #print("Item already in the DB",doc.page_content[:100].replace("\n"," "))
LSDOCS = vectordb.get()["documents"]
vectordb.persist()

2039 docs to add
OPENAI_API_KEY is ready
Continue on the DB
88 elements already stored.
Already 88 documents.
Adding 2039 documents.


/tmp/ipykernel_28908/1336002908.py:25: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=base_path,embedding_function=embeddings)
